# Visualization: Trading Session

In [1]:
import pandas as pd
import numpy as np

import altair as alt
import seaborn as sns

### 1. Define parameters and Load model

In [24]:
from trading_bot.agent import Agent
stock = 'BAJAJ-AUTO'
model_name = '{}_t-dqn_40'.format(stock)
# test_stock = 'data/Nifty50/Split/{}_2019.csv'.format(stock)
test_stock = 'data/{}_2020.csv'.format(stock)
window_size = 10
debug = True

agent = Agent(window_size, pretrained=True, model_name=model_name)

### 2. Load test data

In [39]:
# read csv into dataframe
df = pd.read_csv(test_stock)
# df = df.iloc[:55]
# filter out the desired features

df = df[['Date', 'Adj Close']]
# rename feature column names
df = df.rename(columns={'Adj Close': 'actual', 'Date': 'date'})
# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates

df.head()

,date,actual
0,2020-02-17,2943.420410
1,2020-02-18,2950.023926
2,2020-02-19,2958.062744
3,2020-02-20,2933.754395
4,2020-02-24,2852.407471


### 3. Running Eval

In [40]:
import logging
import coloredlogs

from trading_bot.utils import show_eval_result, switch_k_backend_device, get_stock_data
from trading_bot.methods import evaluate_model

coloredlogs.install(level='DEBUG')
switch_k_backend_device()

test_data = get_stock_data(test_stock)
initial_offset = test_data[1] - test_data[0]

test_result, history = evaluate_model(agent, test_data, window_size, debug)
show_eval_result(model_name, test_result, initial_offset)

2021-02-16 13:33:57 Tornado root[8384] DEBUG switching to TensorFlow for CPU
2021-02-16 13:33:57 Tornado root[8384] DEBUG Buy at: $2943.42
2021-02-16 13:33:57 Tornado root[8384] DEBUG Sell at: $2950.02 | Position: +$6.60
2021-02-16 13:33:57 Tornado root[8384] DEBUG Buy at: $2820.73
2021-02-16 13:33:57 Tornado root[8384] DEBUG Buy at: $2819.96
2021-02-16 13:33:57 Tornado root[8384] DEBUG Buy at: $2810.39
2021-02-16 13:33:57 Tornado root[8384] DEBUG Buy at: $2672.20
2021-02-16 13:33:57 Tornado root[8384] DEBUG Buy at: $2711.50
2021-02-16 13:33:58 Tornado root[8384] DEBUG Sell at: $2573.35 | Position: -$247.38
2021-02-16 13:33:58 Tornado root[8384] DEBUG Sell at: $2357.10 | Position: -$462.86
2021-02-16 13:33:58 Tornado root[8384] DEBUG Sell at: $2339.55 | Position: -$470.84
2021-02-16 13:33:58 Tornado root[8384] DEBUG Sell at: $2270.45 | Position: -$401.75
2021-02-16 13:33:58 Tornado root[8384] DEBUG Sell at: $2242.30 | Position: -$469.20
2021-02-16 13:33:58 Tornado root[8384] DEBUG Buy 

### 4. Visualize

In [41]:
def visualize(df, history, title="trading session"):
    # add history to dataframe
    position = [history[0][0]] + [x[0] for x in history]
    actions = ['HOLD'] + [x[1] for x in history]
    df['position'] = position
    df['action'] = actions
    
    # specify y-axis scale for stock prices|
    scale = alt.Scale(domain=(min(min(df['actual']), min(df['position'])) - 50, max(max(df['actual']), max(df['position'])) + 50), clamp=True)
    
    # plot a line chart for stock positions
    actual = alt.Chart(df).mark_line(
        color='green',
        opacity=0.5
    ).encode(
        x='date:T',
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale)
    ).interactive(
        bind_y=False
    )
    
    # plot the BUY and SELL actions as points
    points = alt.Chart(df).transform_filter(
        alt.datum.action != 'HOLD'
    ).mark_point(
        filled=True
    ).encode(
        x=alt.X('date:T', axis=alt.Axis(title='Date')),
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale),
        color='action'
    ).interactive(bind_y=False)

    # merge the two charts
    chart = alt.layer(actual, points, title=title).properties(height=300, width=1000)
    
    return chart

In [42]:
chart = visualize(df, history, title=test_stock)
chart

alt.LayerChart(...)

In [59]:
cap = [0]
inv = 0
ret = 0
b = 0
for i in range(len(df)):
    if df.iloc[i]['action']=='BUY' and not np.isnan(df.iloc[i]['actual']):
        cap.append(cap[-1]+df.iloc[i]['actual'])
        inv+=df.iloc[i]['actual']
        b+=1
    if df.iloc[i]['action']=='SELL' and b>0 and not np.isnan(df.iloc[i]['actual']):
        cap.append(cap[-1]-df.iloc[i]['actual'])
        ret += df.iloc[i]['actual']
        print(df.iloc[i]['actual'])
        print(i)
#         print(ret)
        b-=1

2958.0627440000003
2
2591.550049
15
2339.550049
18
2270.449951
19
2169.0
20
1935.550049
23
2051.100098
30
2430.649902
42
2505.75
44
2442.149902
48
2696.0
55
2702.649902
56
2663.600098
57
2477.449951
58
2783.399902
69
2776.300049
71
2817.25
84
2820.449951
85
2879.350098
90
2995.0
101
3059.800049
124
3039.350098
127
2880.550049
134
2910.149902
139
2914.699951
140
2916.350098
141
2937.350098
143
3037.949951
144
3019.199951
145
2986.699951
147
2986.199951
149
2957.399902
151
2966.350098
152
2897.5
153
2881.100098
154
3077.399902
158
3054.149902
163
3018.0
168
2895.149902
171
2886.899902
175
3017.449951
184
3319.550049
200
3414.6999509999996
214
3448.149902
216
3600.899902
226
3576.0500490000004
227
4223.5
241
4154.899902
244


In [60]:
type(df.iloc[186].actual)


numpy.float64

In [61]:
req_cap = max(cap)
req_cap

34742.31664999999

In [62]:
prof = ret+(df['action'].value_counts()['BUY']-df['action'].value_counts()['SELL'])*df.iloc[-1]['actual']-inv

In [63]:
ret

139382.71215600002

In [64]:
(prof/ req_cap)*100

-14.559240769570339

In [22]:
df['action'].value_counts()

HOLD    106
BUY      76
SELL     68
Name: action, dtype: int64

In [23]:
list(full['action'].iloc[:len(df)]) == list(df['action'])

NameError: name 'full' is not defined

In [88]:
full

,date,actual,position,action
0,2018-01-01,1715.731567,1715.731567,HOLD
1,2018-01-02,1714.737305,1715.731567,BUY
2,2018-01-03,1705.440918,1714.737305,SELL
3,2018-01-04,1743.918579,1705.440918,BUY
4,2018-01-05,1804.070801,1743.918579,HOLD
5,2018-01-08,1826.292358,1804.070801,SELL
6,2018-01-09,1809.042114,1826.292358,HOLD
7,2018-01-10,1791.841553,1809.042114,HOLD
8,2018-01-11,1770.117065,1791.841553,BUY
9,2018-01-12,1759.926025,1770.117065,BUY
